In [2]:
import modeling_functions_script as md

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation

In [3]:
# Dataset read
dataset = pd.read_csv('./data.csv' , sep= ';')

In [7]:
# Params lists
pixel = 100 # Image shape reduction to 32 x 32

pool_sizes_1 = [(3,3) , (5,5)]
pool_sizes_2 = [(3,3) , (5,5)]
pool_sizes_3 = [(3,3) , (5,5)]

n_neurons = [400, 500, 600]

epochs = [20 , 30 , 40 , 50]
batchs = [32 , 64 , 128]

# Results lists

model_list = []
loss_list = []
train_accuracy_list = []
val_accuracy_list = []
test_loss_list = []
test_accuracy_list = []


In [4]:
image_list = md.read_images(dataset , 100)

In [5]:
image_list[0].shape

(100, 100, 3)

In [8]:
# Train, validation and test data
X_train , X_val , X_test , y_train , y_val , y_test = md.train_test(dataset , image_list)

for pool_size_1 in pool_sizes_1:
    for pool_size_2 in pool_sizes_2:
        for pool_size_3 in pool_sizes_3:
            for n in n_neurons:
                model = md.model_params(pixel , pool_size_1 , pool_size_2 , pool_size_3 , n)

                for epoch in epochs:
                    for batch in batchs:

                        model_list.append((pixel , pool_size_1 , pool_size_2 , pool_size_3 , n , epoch , batch))
                        history , trained_model = md.model_training(model , epoch , batch , X_train , y_train , X_val , y_val)

                        history_df = pd.DataFrame(history.history)
                        loss_list.append(history_df['loss'][epoch-1])
                        train_accuracy_list.append(history_df['accuracy'][epoch-1])
                        val_accuracy_list.append(history_df['val_accuracy'][epoch-1])
                        test_results = trained_model.evaluate(X_test, y_test)
                        test_loss_list.append(test_results[0])
                        test_accuracy_list.append(test_results[1])

-----------Modelo ==>  100 (3, 3) (3, 3) (3, 3) 400 ---------------
Epoch 1/20
94/94 [==============================] - 9s 89ms/step - loss: 0.8926 - accuracy: 0.5413 - val_loss: 0.8132 - val_accuracy: 0.6167
Epoch 2/20
94/94 [==============================] - 8s 88ms/step - loss: 0.6757 - accuracy: 0.6687 - val_loss: 0.6210 - val_accuracy: 0.7267
Epoch 3/20
94/94 [==============================] - 8s 90ms/step - loss: 0.5998 - accuracy: 0.7163 - val_loss: 0.5465 - val_accuracy: 0.7717
Epoch 4/20
94/94 [==============================] - 9s 91ms/step - loss: 0.5238 - accuracy: 0.7783 - val_loss: 0.5018 - val_accuracy: 0.7833
Epoch 5/20
94/94 [==============================] - 9s 93ms/step - loss: 0.4867 - accuracy: 0.7933 - val_loss: 0.4507 - val_accuracy: 0.8017
Epoch 6/20
94/94 [==============================] - 9s 95ms/step - loss: 0.4731 - accuracy: 0.8040 - val_loss: 0.4717 - val_accuracy: 0.8117
Epoch 7/20
94/94 [==============================] - 9s 98ms/step - loss: 0.4325 - accu

KeyboardInterrupt: 

In [11]:
print(len(model_list))
print(len(loss_list))
print(len(train_accuracy_list))
print(len(val_accuracy_list))
print(len(test_loss_list))
print(len(test_accuracy_list))

43
42
42
42
42
42


In [14]:
model_list = model_list[:-1]
print(len(model_list))

42


In [1]:
model_index = pd.MultiIndex.from_tuples(model_list)
results_array = np.array([loss_list , train_accuracy_list , val_accuracy_list , test_loss_list , test_accuracy_list]).transpose()
results_df = pd.DataFrame(results_array , index=model_index , columns= ['train loss' , 'train accuracy' , 'val accuracy' , 'test loss' , 'test accuracy'])
results_df.to_csv('./modeling_results/modeling_100p_results_df.csv' , sep= ';')

NameError: name 'pd' is not defined

In [16]:
results_df.max()

train loss        0.322265
train accuracy    0.997667
val accuracy      0.886667
test loss         0.735381
test accuracy     0.912222
dtype: float64